# Werkcollege-opdrachten Week 1.2

## Voorbereiding

Importeer in het codeblok hieronder de packages die worden gebruikt om data in te lezen. Geef er ook de gebruikelijke aliassen aan.<br>
N.B.: de 2 reeds geschreven coderegels zorgen ervoor dat eventuele warnings, die code-outputs lelijker maken, uitgezet worden.

In [1]:

import warnings
warnings.simplefilter('ignore')

Zet de volgende bestanden in een makkelijk terug te vinden map:
- go_sales_train.sqlite
- go_crm_train.sqlite
- go_staff_train.sqlite

Bestudeer de bovenste 3 bestanden in DB Browser (SQLite), <a href="https://sqlitebrowser.org/dl/">hier</a> te downloaden. Wat valt je op qua datatypen?<br>
Alle datatypen zijn Integers of Strings, geen andere (niet primitieve) datatypen

## Databasetabel inlezen

Creëer een databaseconnectie met het bestand go_sales_train.sqlite.

In [1]:
import sqlite3
con = sqlite3.connect("go_sales_train.sqlite")
con

<b>Let goed op:</b><br>
Als je per ongeluk een verkeerde bestandsnaam ingeeft, maakt Python zélf een leeg databasebestand aan! Er ontstaat dan dus een nieuwe .sqlite, en dat is nadrukkelijk <u>niet de bedoeling.</u>

Gebruik de onderstaande sql_query om te achterhalen welke databasetabellen in go_sales_train zitten.

In [9]:
sql_query = "SELECT name FROM sqlite_master WHERE type='table';"
cursor = con.cursor()
tabellen = cursor.execute(sql_query)
tabellen = tabellen.fetchall()
print(tabellen)

[('country',), ('order_details',), ('order_header',), ('order_method',), ('product',), ('product_line',), ('product_type',), ('retailer_site',), ('return_reason',), ('returned_item',), ('sales_branch',), ('sales_staff',)]


Krijg je lege output? Dan heb je per ongeluk een leeg  databasebestand (.sqlite) aangemaakt.<br>
Lees de informatie onder het kopje <u>Let goed op:</u> hierboven nog eens goed door.

Gebruik de gecreëerde databaseconnectie om de resultaten van de volgende query in een DataFrame te zetten:<br>
*SELECT * FROM sales_staff* 

In [21]:
import pandas as pd

sql_query = "SELECT * FROM sales_staff"
sales_staff = pd.read_sql_query(sql_query, con)
sales_staff

,SALES_STAFF_CODE,FIRST_NAME,LAST_NAME,POSITION_EN,WORK_PHONE,EXTENSION,FAX,EMAIL,DATE_HIRED,SALES_BRANCH_CODE
0,4,Denis,Pagé,Branch Manager,+33 1 68 94 52 20,325.0,+33 1 68 94 56 60,DPage@grtd123.com,1996-11-03,6
1,5,Élizabeth,Michel,Level 3 Sales Representative,+33 1 68 94 52 20,336.0,+33 1 68 94 56 60,EMichel@grtd123.com,1995-06-08,6
2,6,Émile,Clermont,Level 1 Sales Representative,+33 1 68 94 52 20,378.0,+33 1 68 94 56 60,EClermont@grtd123.com,1998-04-07,6
3,7,Étienne,Jauvin,Level 2 Sales Representative,+33 1 68 94 52 20,398.0,+33 1 68 94 56 60,EJauvin@grtd123.com,1997-08-16,6
4,12,Elsbeth,Wiesinger,Level 2 Sales Representative,+(49) 40 663 1990,1818.0,+(49) 40 663 4571,EWiesinger@grtd123.com,1997-02-27,13
...,...,...,...,...,...,...,...,...,...,...
97,120,Giele,Laermans,Level 1 Sales Representative,+32 16 20.73.21,1340.0,+32 16 20.73.32,GLaermans@grtd123.com,1999-11-15,38
98,121,François,De Crée,Level 1 Sales Representative,+32 16 20.73.21,1642.0,+32 16 20.73.32,FDecree@grtd123.com,1999-12-01,38
99,122,Yvette,Lattrez,Level 3 Sales Representative,+32 16 20.73.21,1633.0,+32 16 20.73.32,YLattrez@grtd123.com,1999-12-09,38
100,123,Willi,Seefelder,Level 2 Sales Representative,+(43) 13 79 56 32,325.0,+(43) 13 79 56 33,WSeefelder@grtd123.com,1998-10-28,39


## Datumkolommen

Zoals je misschien al hebt gezien in DB Browser, zijn datums als TEXT opgeslagen, en niet als DATE, DATETIME o.i.d. Hier moeten we dus nog even "typische datumkolommen" van maken. Dat doen we met de volgende code:

In [18]:
sales_staff['DATE_HIRED'] = pd.to_datetime(sales_staff['DATE_HIRED'])
sales_staff.dtypes

SALES_STAFF_CODE              int64
FIRST_NAME                   object
LAST_NAME                    object
POSITION_EN                  object
WORK_PHONE                   object
EXTENSION                   float64
FAX                          object
EMAIL                        object
DATE_HIRED           datetime64[ns]
SALES_BRANCH_CODE             int64
dtype: object

Als we hier het jaar uit willen halen, schrijven we:

In [19]:
pd.DatetimeIndex(sales_staff['DATE_HIRED']).quarter

Index([4, 2, 2, 3, 1, 2, 1, 1, 3, 3,
       ...
       3, 1, 4, 1, 3, 4, 4, 4, 4, 1],
      dtype='int32', name='DATE_HIRED', length=102)

Deze zelfde syntax is te gebruiken voor het extraheren van kwartalen, maanden, weken en dagen. Probeer het maar eens!

## DataFrames uitsplitsen en opbouwen met Series

De volgende 5 kolommen hebben betrekking op de contactdetails van elke medewerker in dit DataFrame:
- SALES_STAFF_CODE
- WORK_PHONE
- EXTENSION
- FAX
- EMAIL

Maak van elk van deze 5 kolommen een serie.

In [27]:
a = pd.Series(sales_staff["SALES_STAFF_CODE"]);
b = pd.Series(sales_staff["WORK_PHONE"]);
c = pd.Series(sales_staff["EXTENSION"]);
d = pd.Series(sales_staff["FAX"]);
e = pd.Series(sales_staff["EMAIL"]);

print(a)
print(b)
print(c)
print(d)
print(e)

0        4
1        5
2        6
3        7
4       12
      ... 
97     120
98     121
99     122
100    123
101    124
Name: SALES_STAFF_CODE, Length: 102, dtype: int64
0      +33 1 68 94 52 20
1      +33 1 68 94 52 20
2      +33 1 68 94 52 20
3      +33 1 68 94 52 20
4      +(49) 40 663 1990
             ...        
97       +32 16 20.73.21
98       +32 16 20.73.21
99       +32 16 20.73.21
100    +(43) 13 79 56 32
101    +(43) 13 79 56 32
Name: WORK_PHONE, Length: 102, dtype: object
0       325.0
1       336.0
2       378.0
3       398.0
4      1818.0
        ...  
97     1340.0
98     1642.0
99     1633.0
100     325.0
101     348.0
Name: EXTENSION, Length: 102, dtype: float64
0      +33 1 68 94 56 60
1      +33 1 68 94 56 60
2      +33 1 68 94 56 60
3      +33 1 68 94 56 60
4      +(49) 40 663 4571
             ...        
97       +32 16 20.73.32
98       +32 16 20.73.32
99       +32 16 20.73.32
100    +(43) 13 79 56 33
101    +(43) 13 79 56 33
Name: FAX, Length: 102, dtype: obje

Zet allevijf gecreëerde series als kolommen naast elkaar in een DataFrame (*contact_details*). Gebruik pd.concat(...)<br>
Hulpvraag: welke waarde geef je aan de axis-parameter?

In [31]:
contact_details = pd.concat([a, b, c, d, e], axis=1)
contact_details

,SALES_STAFF_CODE,WORK_PHONE,EXTENSION,FAX,EMAIL
0,4,+33 1 68 94 52 20,325.0,+33 1 68 94 56 60,DPage@grtd123.com
1,5,+33 1 68 94 52 20,336.0,+33 1 68 94 56 60,EMichel@grtd123.com
2,6,+33 1 68 94 52 20,378.0,+33 1 68 94 56 60,EClermont@grtd123.com
3,7,+33 1 68 94 52 20,398.0,+33 1 68 94 56 60,EJauvin@grtd123.com
4,12,+(49) 40 663 1990,1818.0,+(49) 40 663 4571,EWiesinger@grtd123.com
...,...,...,...,...,...
97,120,+32 16 20.73.21,1340.0,+32 16 20.73.32,GLaermans@grtd123.com
98,121,+32 16 20.73.21,1642.0,+32 16 20.73.32,FDecree@grtd123.com
99,122,+32 16 20.73.21,1633.0,+32 16 20.73.32,YLattrez@grtd123.com
100,123,+(43) 13 79 56 32,325.0,+(43) 13 79 56 33,WSeefelder@grtd123.com


## Series en DataFrames maken vanuit lists en dictionaries

Met .head(*getal*) kan je de bovenste *getal* rijen van een tabel selecteren.<br>
Selecteer op deze manier de bovenste 5 rijen van *contact_details*.<br>
Sla dit resultaat op in een nieuw DataFrame.

In [32]:
nieuwe_data = contact_details.head(5)

nieuwe_data

,SALES_STAFF_CODE,WORK_PHONE,EXTENSION,FAX,EMAIL
0,4,+33 1 68 94 52 20,325.0,+33 1 68 94 56 60,DPage@grtd123.com
1,5,+33 1 68 94 52 20,336.0,+33 1 68 94 56 60,EMichel@grtd123.com
2,6,+33 1 68 94 52 20,378.0,+33 1 68 94 56 60,EClermont@grtd123.com
3,7,+33 1 68 94 52 20,398.0,+33 1 68 94 56 60,EJauvin@grtd123.com
4,12,+(49) 40 663 1990,1818.0,+(49) 40 663 4571,EWiesinger@grtd123.com


Aan deze 10 rijen met contactdetails willen we 3 kolommen toevoegen: 'FIRST_LANGUAGE', 'SECOND_LANGUAGE' & 'THIRD_LANGUAGE'.<br>
Iedereens 'First Language' is Engels, afgekort 'EN'. Maak een lijst waarin 5x 'EN' staat.<br>
Converteer deze lijst vervolgens naar een serie en voeg deze horizontaal samen met het resultaat van de vorige opdracht.<br>
Vergeet niet een passende naam te geven aan de nieuw ontstane kolom.

In [43]:
engels = ['EN', "EN", "EN", "EN", "EN"]
engels = pd.Series(engels, name='FIRST_LANGUAGE')

inclusief_taal = pd.concat([nieuwe_data, engels], axis=1)
inclusief_taal

,SALES_STAFF_CODE,WORK_PHONE,EXTENSION,FAX,EMAIL,FIRST_LANGUAGE
0,4,+33 1 68 94 52 20,325.0,+33 1 68 94 56 60,DPage@grtd123.com,EN
1,5,+33 1 68 94 52 20,336.0,+33 1 68 94 56 60,EMichel@grtd123.com,EN
2,6,+33 1 68 94 52 20,378.0,+33 1 68 94 56 60,EClermont@grtd123.com,EN
3,7,+33 1 68 94 52 20,398.0,+33 1 68 94 56 60,EJauvin@grtd123.com,EN
4,12,+(49) 40 663 1990,1818.0,+(49) 40 663 4571,EWiesinger@grtd123.com,EN


Maak nu de tweede kolom ('SECOND_LANGUAGE'). Maak daarvoor een dictionary, met daarin...
- Als keys: alle indexen uit het resultaat van het vorige codeblok.
- Als values: bij de eerste 3 elementen 'FR' (Frankrijk), bij de laatste 2 'DE' (Duitsland).

Maak vervolgens ook hier weer een serie van en voeg ook deze weer horizontaal samen met het rsultaat van de vorige opdracht.<br>
Vergeet niet een passende naam te geven aan de nieuw ontstane kolom.

In [51]:
frans_duijts = {0: 'FR', 1: 'FR', 2: 'FR', 3: 'DE', 4: 'DE'}
frans_duijts = pd.Series(frans_duijts, name='SECOND_LANGUAGE')

inclusief_taal = pd.concat([inclusief_taal, frans_duijts], axis=1)
inclusief_taal

,SALES_STAFF_CODE,WORK_PHONE,EXTENSION,FAX,EMAIL,FIRST_LANGUAGE,SECOND_LANGUAGE
0,4.0,+33 1 68 94 52 20,325.0,+33 1 68 94 56 60,DPage@grtd123.com,EN,FR
1,5.0,+33 1 68 94 52 20,336.0,+33 1 68 94 56 60,EMichel@grtd123.com,EN,FR
2,6.0,+33 1 68 94 52 20,378.0,+33 1 68 94 56 60,EClermont@grtd123.com,EN,FR
3,7.0,+33 1 68 94 52 20,398.0,+33 1 68 94 56 60,EJauvin@grtd123.com,EN,DE
4,12.0,+(49) 40 663 1990,1818.0,+(49) 40 663 4571,EWiesinger@grtd123.com,EN,DE
THIRD_LANGUAGE,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Maak ten slotte de derde kolom ('THIRD_LANGUAGE') door een dictionary te maken met daarin...
- Als key: de naam van de nieuwe kolom. Zie je het verschil qua keys met de vorige opdracht?
- Als waarde: een lijst met daarin 'NL', 'NL', 'JPN', 'JPN', 'KOR'.

Converteer deze dictionary nu naar een DataFrame en voeg deze horizontaal samen met het resultaat van de vorige opdracht.<br>
Waarom hoef je hierna de nieuw ontstane kolom geen passende naam meer te geven?

In [55]:
nl_jpn_kor = {
    'THIRD_LANGUAGE': [
        'NL',
        'NL',
        "JPN",
        "JPN",
        "KOR"
    ]
}

nl_jpn_kor = pd.DataFrame(nl_jpn_kor)


inclusief_taal = pd.concat([inclusief_taal, nl_jpn_kor], axis=1)
inclusief_taal

,SALES_STAFF_CODE,WORK_PHONE,EXTENSION,FAX,EMAIL,FIRST_LANGUAGE,SECOND_LANGUAGE,0,0,THIRD_LANGUAGE
0,4.0,+33 1 68 94 52 20,325.0,+33 1 68 94 56 60,DPage@grtd123.com,EN,FR,NaN,NaN,NL
1,5.0,+33 1 68 94 52 20,336.0,+33 1 68 94 56 60,EMichel@grtd123.com,EN,FR,NaN,NaN,NL
2,6.0,+33 1 68 94 52 20,378.0,+33 1 68 94 56 60,EClermont@grtd123.com,EN,FR,NaN,NaN,JPN
3,7.0,+33 1 68 94 52 20,398.0,+33 1 68 94 56 60,EJauvin@grtd123.com,EN,DE,NaN,NaN,JPN
4,12.0,+(49) 40 663 1990,1818.0,+(49) 40 663 4571,EWiesinger@grtd123.com,EN,DE,NaN,NaN,KOR
THIRD_LANGUAGE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{KOR, JPN, NL}","[NL, NL, JPN, JPN, KOR]",NaN


## Data toevoegen

### Rijen

Gebruik de originele databasetabel *sales_staff*.<br>
Voeg een extra rij toe met eigen invulling. Zorg ervoor dat de index netjes doorloopt.<br>
Hulpvraag: welke waarde geef je nu aan axis?

In [62]:
nieuwe_medewerker = {
    'SALES_STAFF_CODE': 23143797,
    'FIRST_NAME': 'Coen',
    'LAST_NAME': 'Vink',
    'POSITION_EN': 'Software Ontwikkelaar',
    'WORK_PHONE': '+31626869897',
    'EXTENSION': 154,
    'FAX': '+31626869897',
    'EMAIL': 'pctcoen@gmail.com',
    'DATE_HIRED': '23-02-2003',
    'SALES_BRANCH_CODE': "21"
}

nieuwe_medewerker_tabel = pd.DataFrame([nieuwe_medewerker]);

totale_tabel = pd.concat([nieuwe_medewerker_tabel, sales_staff], axis=0)

totale_tabel

,SALES_STAFF_CODE,FIRST_NAME,LAST_NAME,POSITION_EN,WORK_PHONE,EXTENSION,FAX,EMAIL,DATE_HIRED,SALES_BRANCH_CODE
0,23143797,Coen,Vink,Software Ontwikkelaar,+31626869897,154.0,+31626869897,pctcoen@gmail.com,23-02-2003,21
0,4,Denis,Pagé,Branch Manager,+33 1 68 94 52 20,325.0,+33 1 68 94 56 60,DPage@grtd123.com,1996-11-03,6
1,5,Élizabeth,Michel,Level 3 Sales Representative,+33 1 68 94 52 20,336.0,+33 1 68 94 56 60,EMichel@grtd123.com,1995-06-08,6
2,6,Émile,Clermont,Level 1 Sales Representative,+33 1 68 94 52 20,378.0,+33 1 68 94 56 60,EClermont@grtd123.com,1998-04-07,6
3,7,Étienne,Jauvin,Level 2 Sales Representative,+33 1 68 94 52 20,398.0,+33 1 68 94 56 60,EJauvin@grtd123.com,1997-08-16,6
...,...,...,...,...,...,...,...,...,...,...
97,120,Giele,Laermans,Level 1 Sales Representative,+32 16 20.73.21,1340.0,+32 16 20.73.32,GLaermans@grtd123.com,1999-11-15,38
98,121,François,De Crée,Level 1 Sales Representative,+32 16 20.73.21,1642.0,+32 16 20.73.32,FDecree@grtd123.com,1999-12-01,38
99,122,Yvette,Lattrez,Level 3 Sales Representative,+32 16 20.73.21,1633.0,+32 16 20.73.32,YLattrez@grtd123.com,1999-12-09,38
100,123,Willi,Seefelder,Level 2 Sales Representative,+(43) 13 79 56 32,325.0,+(43) 13 79 56 33,WSeefelder@grtd123.com,1998-10-28,39


### Kolommen

Voeg een kolom *FULL_NAME* toe die de waarden van *FIRST_NAME* en *LAST_NAME* achter elkaar zet, gescheiden door een spatie.

In [63]:
sales_staff['FULL_NAME'] = sales_staff['FIRST_NAME'] + ' ' + sales_staff['LAST_NAME']
sales_staff


,SALES_STAFF_CODE,FIRST_NAME,LAST_NAME,POSITION_EN,WORK_PHONE,EXTENSION,FAX,EMAIL,DATE_HIRED,SALES_BRANCH_CODE,FULL_NAME
0,4,Denis,Pagé,Branch Manager,+33 1 68 94 52 20,325.0,+33 1 68 94 56 60,DPage@grtd123.com,1996-11-03,6,Denis Pagé
1,5,Élizabeth,Michel,Level 3 Sales Representative,+33 1 68 94 52 20,336.0,+33 1 68 94 56 60,EMichel@grtd123.com,1995-06-08,6,Élizabeth Michel
2,6,Émile,Clermont,Level 1 Sales Representative,+33 1 68 94 52 20,378.0,+33 1 68 94 56 60,EClermont@grtd123.com,1998-04-07,6,Émile Clermont
3,7,Étienne,Jauvin,Level 2 Sales Representative,+33 1 68 94 52 20,398.0,+33 1 68 94 56 60,EJauvin@grtd123.com,1997-08-16,6,Étienne Jauvin
4,12,Elsbeth,Wiesinger,Level 2 Sales Representative,+(49) 40 663 1990,1818.0,+(49) 40 663 4571,EWiesinger@grtd123.com,1997-02-27,13,Elsbeth Wiesinger
...,...,...,...,...,...,...,...,...,...,...,...
97,120,Giele,Laermans,Level 1 Sales Representative,+32 16 20.73.21,1340.0,+32 16 20.73.32,GLaermans@grtd123.com,1999-11-15,38,Giele Laermans
98,121,François,De Crée,Level 1 Sales Representative,+32 16 20.73.21,1642.0,+32 16 20.73.32,FDecree@grtd123.com,1999-12-01,38,François De Crée
99,122,Yvette,Lattrez,Level 3 Sales Representative,+32 16 20.73.21,1633.0,+32 16 20.73.32,YLattrez@grtd123.com,1999-12-09,38,Yvette Lattrez
100,123,Willi,Seefelder,Level 2 Sales Representative,+(43) 13 79 56 32,325.0,+(43) 13 79 56 33,WSeefelder@grtd123.com,1998-10-28,39,Willi Seefelder


## Data wijzigen

### Datatypen

Door het attribuut .dtypes van een DataFrame op te vragen krijg je een serie die per kolom het datatype weergeeft. Doe dit bij de originele databasetabel *sales_staff*

In [65]:
sales_staff.dtypes

SALES_STAFF_CODE       int64
FIRST_NAME            object
LAST_NAME             object
POSITION_EN           object
WORK_PHONE            object
EXTENSION            float64
FAX                   object
EMAIL                 object
DATE_HIRED            object
SALES_BRANCH_CODE      int64
FULL_NAME             object
dtype: object

Hier valt op dat elke kolom het datatype 'object' heeft: Python leest dus alles als string. Wiskundige operaties zijn hierdoor niet mogelijk.<br>
We kunnen proberen om kolommen met getallen, bijvoorbeeld de 'extension', te converteren naar een int. Zie onderstaande code.

In [68]:
sales_staff['EXTENSION'] = sales_staff['EXTENSION'].astype(int)
sales_staff.dtypes

IntCastingNaNError: Cannot convert non-finite values (NA or inf) to integer

Dit lukt echter niet, omdat er in de kolom 'EXTENSION' lege waarden zitten die niet geconverteerd kunnen worden naar een int.<br>
Wél kan je deze naar een float converteren, zie onderstaande code:

In [67]:
sales_staff['EXTENSION'] = sales_staff['EXTENSION'].astype(float)
sales_staff.dtypes

SALES_STAFF_CODE       int64
FIRST_NAME            object
LAST_NAME             object
POSITION_EN           object
WORK_PHONE            object
EXTENSION            float64
FAX                   object
EMAIL                 object
DATE_HIRED            object
SALES_BRANCH_CODE      int64
FULL_NAME             object
dtype: object

Als we in de rij van 'EXTENSION' kijken zien we dat de conversie van het datatype nu is gelukt.<br>
Dit is <b>randvoorwaardelijk</b> voor het uitvoeren van wiskundige operaties.<br>

### Rijen

Zorg er nu voor dat bij alle extensions 1 wordt opgeteld.

Elke 'Branch Manager' wordt nu 'General Manager'. Schrijf code zodat deze wijziging doorgevoerd wordt in het DataFrame.

In [69]:
nieuwe_frame = pd.DataFrame(sales_staff).replace("Branch Manager", "General Manager")
nieuwe_frame

,SALES_STAFF_CODE,FIRST_NAME,LAST_NAME,POSITION_EN,WORK_PHONE,EXTENSION,FAX,EMAIL,DATE_HIRED,SALES_BRANCH_CODE,FULL_NAME
0,4,Denis,Pagé,General Manager,+33 1 68 94 52 20,325.0,+33 1 68 94 56 60,DPage@grtd123.com,1996-11-03,6,Denis Pagé
1,5,Élizabeth,Michel,Level 3 Sales Representative,+33 1 68 94 52 20,336.0,+33 1 68 94 56 60,EMichel@grtd123.com,1995-06-08,6,Élizabeth Michel
2,6,Émile,Clermont,Level 1 Sales Representative,+33 1 68 94 52 20,378.0,+33 1 68 94 56 60,EClermont@grtd123.com,1998-04-07,6,Émile Clermont
3,7,Étienne,Jauvin,Level 2 Sales Representative,+33 1 68 94 52 20,398.0,+33 1 68 94 56 60,EJauvin@grtd123.com,1997-08-16,6,Étienne Jauvin
4,12,Elsbeth,Wiesinger,Level 2 Sales Representative,+(49) 40 663 1990,1818.0,+(49) 40 663 4571,EWiesinger@grtd123.com,1997-02-27,13,Elsbeth Wiesinger
...,...,...,...,...,...,...,...,...,...,...,...
97,120,Giele,Laermans,Level 1 Sales Representative,+32 16 20.73.21,1340.0,+32 16 20.73.32,GLaermans@grtd123.com,1999-11-15,38,Giele Laermans
98,121,François,De Crée,Level 1 Sales Representative,+32 16 20.73.21,1642.0,+32 16 20.73.32,FDecree@grtd123.com,1999-12-01,38,François De Crée
99,122,Yvette,Lattrez,Level 3 Sales Representative,+32 16 20.73.21,1633.0,+32 16 20.73.32,YLattrez@grtd123.com,1999-12-09,38,Yvette Lattrez
100,123,Willi,Seefelder,Level 2 Sales Representative,+(43) 13 79 56 32,325.0,+(43) 13 79 56 33,WSeefelder@grtd123.com,1998-10-28,39,Willi Seefelder


### Kolommen

Verander de kolomnaam 'POSITION_EN' naar 'POSITION'.

In [73]:
nieuwe_frame = pd.DataFrame(nieuwe_frame).rename({"POSITION_EN": "POSITION"}, axis=1)
nieuwe_frame

,SALES_STAFF_CODE,FIRST_NAME,LAST_NAME,POSITION,WORK_PHONE,EXTENSION,FAX,EMAIL,DATE_HIRED,SALES_BRANCH_CODE,FULL_NAME
0,4,Denis,Pagé,General Manager,+33 1 68 94 52 20,325.0,+33 1 68 94 56 60,DPage@grtd123.com,1996-11-03,6,Denis Pagé
1,5,Élizabeth,Michel,Level 3 Sales Representative,+33 1 68 94 52 20,336.0,+33 1 68 94 56 60,EMichel@grtd123.com,1995-06-08,6,Élizabeth Michel
2,6,Émile,Clermont,Level 1 Sales Representative,+33 1 68 94 52 20,378.0,+33 1 68 94 56 60,EClermont@grtd123.com,1998-04-07,6,Émile Clermont
3,7,Étienne,Jauvin,Level 2 Sales Representative,+33 1 68 94 52 20,398.0,+33 1 68 94 56 60,EJauvin@grtd123.com,1997-08-16,6,Étienne Jauvin
4,12,Elsbeth,Wiesinger,Level 2 Sales Representative,+(49) 40 663 1990,1818.0,+(49) 40 663 4571,EWiesinger@grtd123.com,1997-02-27,13,Elsbeth Wiesinger
...,...,...,...,...,...,...,...,...,...,...,...
97,120,Giele,Laermans,Level 1 Sales Representative,+32 16 20.73.21,1340.0,+32 16 20.73.32,GLaermans@grtd123.com,1999-11-15,38,Giele Laermans
98,121,François,De Crée,Level 1 Sales Representative,+32 16 20.73.21,1642.0,+32 16 20.73.32,FDecree@grtd123.com,1999-12-01,38,François De Crée
99,122,Yvette,Lattrez,Level 3 Sales Representative,+32 16 20.73.21,1633.0,+32 16 20.73.32,YLattrez@grtd123.com,1999-12-09,38,Yvette Lattrez
100,123,Willi,Seefelder,Level 2 Sales Representative,+(43) 13 79 56 32,325.0,+(43) 13 79 56 33,WSeefelder@grtd123.com,1998-10-28,39,Willi Seefelder


## Data verwijderen

### Rijen

De medewerkers op indexen 99, 100 en 101 hebben helaas ontslag genomen.<br>
Verwijder de bijbehorende rijen uit het DataFrame. Gebruik slechts één keer de .drop()-methode.

In [74]:
nieuwe_frame = nieuwe_frame.drop([99,100, 101], axis=0)
nieuwe_frame

,SALES_STAFF_CODE,FIRST_NAME,LAST_NAME,POSITION,WORK_PHONE,EXTENSION,FAX,EMAIL,DATE_HIRED,SALES_BRANCH_CODE,FULL_NAME
0,4,Denis,Pagé,General Manager,+33 1 68 94 52 20,325.0,+33 1 68 94 56 60,DPage@grtd123.com,1996-11-03,6,Denis Pagé
1,5,Élizabeth,Michel,Level 3 Sales Representative,+33 1 68 94 52 20,336.0,+33 1 68 94 56 60,EMichel@grtd123.com,1995-06-08,6,Élizabeth Michel
2,6,Émile,Clermont,Level 1 Sales Representative,+33 1 68 94 52 20,378.0,+33 1 68 94 56 60,EClermont@grtd123.com,1998-04-07,6,Émile Clermont
3,7,Étienne,Jauvin,Level 2 Sales Representative,+33 1 68 94 52 20,398.0,+33 1 68 94 56 60,EJauvin@grtd123.com,1997-08-16,6,Étienne Jauvin
4,12,Elsbeth,Wiesinger,Level 2 Sales Representative,+(49) 40 663 1990,1818.0,+(49) 40 663 4571,EWiesinger@grtd123.com,1997-02-27,13,Elsbeth Wiesinger
...,...,...,...,...,...,...,...,...,...,...,...
94,117,Frank,Jever,Level 3 Sales Representative,+(41) 17 12 13 14,1154.0,+(41) 17 12 13 20,FJever@grtd123.com,1999-10-18,37,Frank Jever
95,118,Gianni,Vertemati,Level 1 Sales Representative,+(41) 17 12 13 14,1254.0,+(41) 17 12 13 20,GVertemati@grtd123.com,2000-02-10,37,Gianni Vertemati
96,119,Gracy,Gellens,General Manager,+32 16 20.73.21,1349.0,+32 16 20.73.32,GGellens@grtd123.com,1999-09-10,38,Gracy Gellens
97,120,Giele,Laermans,Level 1 Sales Representative,+32 16 20.73.21,1340.0,+32 16 20.73.32,GLaermans@grtd123.com,1999-11-15,38,Giele Laermans


### Kolommen

Faxen zijn inmiddels ouderwets: niemand gebruikt zijn/haar faxnummer nog.<br>
Verwijder de bijbehorende kolom uit het DataFrame.

In [75]:
nieuwe_frame = nieuwe_frame.drop('FAX', axis=1);
nieuwe_frame

,SALES_STAFF_CODE,FIRST_NAME,LAST_NAME,POSITION,WORK_PHONE,EXTENSION,EMAIL,DATE_HIRED,SALES_BRANCH_CODE,FULL_NAME
0,4,Denis,Pagé,General Manager,+33 1 68 94 52 20,325.0,DPage@grtd123.com,1996-11-03,6,Denis Pagé
1,5,Élizabeth,Michel,Level 3 Sales Representative,+33 1 68 94 52 20,336.0,EMichel@grtd123.com,1995-06-08,6,Élizabeth Michel
2,6,Émile,Clermont,Level 1 Sales Representative,+33 1 68 94 52 20,378.0,EClermont@grtd123.com,1998-04-07,6,Émile Clermont
3,7,Étienne,Jauvin,Level 2 Sales Representative,+33 1 68 94 52 20,398.0,EJauvin@grtd123.com,1997-08-16,6,Étienne Jauvin
4,12,Elsbeth,Wiesinger,Level 2 Sales Representative,+(49) 40 663 1990,1818.0,EWiesinger@grtd123.com,1997-02-27,13,Elsbeth Wiesinger
...,...,...,...,...,...,...,...,...,...,...
94,117,Frank,Jever,Level 3 Sales Representative,+(41) 17 12 13 14,1154.0,FJever@grtd123.com,1999-10-18,37,Frank Jever
95,118,Gianni,Vertemati,Level 1 Sales Representative,+(41) 17 12 13 14,1254.0,GVertemati@grtd123.com,2000-02-10,37,Gianni Vertemati
96,119,Gracy,Gellens,General Manager,+32 16 20.73.21,1349.0,GGellens@grtd123.com,1999-09-10,38,Gracy Gellens
97,120,Giele,Laermans,Level 1 Sales Representative,+32 16 20.73.21,1340.0,GLaermans@grtd123.com,1999-11-15,38,Giele Laermans
